# Data Wrangling: Clean, Transform, Merge, Reshape

## Combining and Merging Data Sets
* pandas.merge - connects rows DataFrames based on one or more keys. This will be familiar to users of SQL or to other relational databases, as it implements database *join* operations
* pandas.concat - glues or stacks together objects along an axis
* combine_first - instance method enables splicing together overlapping data to fill in missing values in one object with values from another

### Database-style DataFrame Merges
*Merge* or *join* operations combine data sets by linking rows using one or more *keys*. These operations are central to relational databases. the `merge` function in pandas is the main entry point for using these algorithms on your data.

In [60]:
# need to import pandas and numpy
import pandas as pd
import numpy as np

In [2]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': xrange(7)})

In [3]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': xrange(3)})

In [4]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [5]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


Perform a `many-to-one` merge situation.

In [6]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


If you don't specify which column to join on. If not specified, `merge` uses the overlapping column names as the keys. Good practice to specify.

In [7]:
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


If the column names are different in each object, you can specify them separately

In [9]:
df1 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': xrange(7)})

In [10]:
df2 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': xrange(3)})

In [11]:
df1

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [12]:
df2

,data2,rkey
0,0,a
1,1,b
2,2,d


In [14]:
pd.merge(df1, df2, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


Perform outer join

In [16]:
pd.merge(df1, df2, left_on='lkey', right_on='rkey', how='outer')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a
6,3,c,NaN,NaN
7,NaN,NaN,2,d


`Many-to-many` joins form the Cartesian product of the rows.

In [22]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': xrange(6)})

In [23]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [24]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': xrange(5)})

In [25]:
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [30]:
many_left = pd.merge(df1, df2, on='key', how='left')

In [35]:
many_left.sort_values('key') # same as many_left.sort_values('key', axis=0)

,data1,key,data2
4,2,a,0
5,2,a,2
7,4,a,0
8,4,a,2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
9,5,b,1
10,5,b,3


In [36]:
many_outer = pd.merge(df1, df2, on='key', how='outer')

In [37]:
many_outer.equals(many_left)

False

In [39]:
many_outer

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


Merge with multiple keys, pass a list of column names

In [44]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})

In [45]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

In [46]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [47]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [48]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,NaN
3,bar,one,3,6
4,bar,two,NaN,7


In [49]:
pd.merge(left, right, on=['key1', 'key2'], how='left')

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,NaN
3,bar,one,3,6


Use suffixes when you have overlapping column names that are not keys

In [50]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [51]:
# more informative
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index

In [54]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': xrange(6)})

In [55]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [56]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [57]:
right1

,group_val
a,3.5
b,7.0


In [58]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [59]:
pd.merge(left1, right1, left_on='key', right_index=True, how='left')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


With hierarchly-indexed data, things are a bit more complicated.

In [61]:
left_h = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                       'key2': [2000, 2001, 2002, 2001, 2002],
                       'data': np.arange(5.)})

In [62]:
left_h

,data,key1,key2
0,0,Ohio,2000
1,1,Ohio,2001
2,2,Ohio,2002
3,3,Nevada,2001
4,4,Nevada,2002


In [65]:
right_h = pd.DataFrame(np.arange(12).reshape((6, 2)), 
                       index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                              [2001, 2000, 2000, 2000, 2001, 2002]],
                       columns=['event1', 'event2'])

In [66]:
right_h

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In this case, you have to indicate multiple columns to merge on as a list (pay attention to the handling of duplicate index values)

In [67]:
pd.merge(left_h, right_h, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0,Ohio,2000,4,5
0,0,Ohio,2000,6,7
1,1,Ohio,2001,8,9
2,2,Ohio,2002,10,11
3,3,Nevada,2001,0,1


In addition to the module-level merge function, DataFrame has an instance method, `join`. When DataFrames have similar indexes and non-overlapping columns.

In [68]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])

In [69]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [71]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14.]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [72]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [73]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1,2,NaN,NaN
b,NaN,NaN,7,8
c,3,4,9,10
d,NaN,NaN,11,12
e,5,6,13,14


In part for legacy reasons (much earlier versions of pandas), DataFrame's `join` method performs a left join on the join keys. It also supports joining the index of the passed DataFrame on one of the columns of the calling DataFrame.

In [75]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


For simple index-on-index merges, you can pass a list of DataFrames to `join` as an alternative to using the more general `concat` function.

In [76]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])

In [77]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,NaN,NaN,7,8
c,3,4,9,10,9,10
e,5,6,13,14,11,12


In [78]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,NaN,NaN,7,8
b,NaN,NaN,7,8,NaN,NaN
c,3,4,9,10,9,10
d,NaN,NaN,11,12,NaN,NaN
e,5,6,13,14,11,12
f,NaN,NaN,NaN,NaN,16,17


### Concatenating Along an Axis

In [79]:
# first, the numpy version
arr = np.arange(12).reshape((3, 4))

In [80]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [83]:
np.concatenate([arr, arr], axis=0) # stack (by row)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [84]:
np.concatenate([arr, arr], axis=1) # place side by side (by column)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

Three Series with no index overlap.

In [85]:
s1 = pd.Series([0, 1], index=['a', 'b'])

In [86]:
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])

In [87]:
s3 = pd.Series([5, 6], index=['f', 'g'])

In [88]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [89]:
# seems not intuitive at first (because of the labels)
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0,NaN,NaN
b,1,NaN,NaN
c,NaN,2,NaN
d,NaN,3,NaN
e,NaN,4,NaN
f,NaN,NaN,5
g,NaN,NaN,6


To intersect the "columns", pass 'join="inner"'.

In [92]:
s4 = pd.concat([s1 * 5, s3])

In [93]:
s4

a    0
b    5
f    5
g    6
dtype: int64

In [94]:
pd.concat([s1, s4], axis=1) # by default, outer

,0,1
a,0,0
b,1,5
f,NaN,5
g,NaN,6


In [95]:
pd.concat([s1, s4], axis=1, join='inner') 

,0,1
a,0,0
b,1,5


In [106]:
# or just specify the axis to join on
pd.concat([s1, s4], axis=1, join_axes=[['a', 'b', 'c', 'e']])

,0,1
a,0,0
b,1,5
c,NaN,NaN
e,NaN,NaN


In [107]:
# create a hierarchical index
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])

In [108]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [109]:
result.unstack()

,a,b,f,g
one,0,1,NaN,NaN
two,0,1,NaN,NaN
three,NaN,NaN,5,6


In the case of combining Series along axis=1, the `keys` become the DataFrame column.

In [110]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0,NaN,NaN
b,1,NaN,NaN
c,NaN,2,NaN
d,NaN,3,NaN
e,NaN,4,NaN
f,NaN,NaN,5
g,NaN,NaN,6


Now, DataFrames

In [111]:
df1 = pd.DataFrame(np.arange(6).reshape((3, 2)),
                   index=['a', 'b', 'c'],
                   columns=['one', 'two'])

In [113]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [116]:
df2 = pd.DataFrame(5 + np.arange(4). reshape((2, 2)),
                   index=['a', 'c'],
                   columns=['three', 'four'])

In [117]:
df2

,three,four
a,5,6
c,7,8


In [119]:
pd.concat([df1, df2]) # by default axis 0 (by row)

,four,one,three,two
a,NaN,0,NaN,1
b,NaN,2,NaN,3
c,NaN,4,NaN,5
a,6,NaN,5,NaN
c,8,NaN,7,NaN


In [120]:
pd.concat([df1, df2], axis=0)

,four,one,three,two
a,NaN,0,NaN,1
b,NaN,2,NaN,3
c,NaN,4,NaN,5
a,6,NaN,5,NaN
c,8,NaN,7,NaN


In [121]:
pd.concat([df1, df2], axis=1)

,one,two,three,four
a,0,1,5,6
b,2,3,NaN,NaN
c,4,5,7,8


In [122]:
# hierarchical index on concatenation index
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1      5    6
b      2   3    NaN  NaN
c      4   5      7    8

In [123]:
pd.concat([df1, df2], axis=0, keys=['level1', 'level2'])

four  one  three  two
level1 a   NaN    0    NaN    1
       b   NaN    2    NaN    3
       c   NaN    4    NaN    5
level2 a     6  NaN      5  NaN
       c     8  NaN      7  NaN

In [124]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1      5    6
b          2   3    NaN  NaN
c          4   5      7    8

In [125]:
pd.concat([df1, df2], ignore_index=True)

,four,one,three,two
0,NaN,0,NaN,1
1,NaN,2,NaN,3
2,NaN,4,NaN,5
3,6,NaN,5,NaN
4,8,NaN,7,NaN


In [126]:
pd.concat([df1, df2], axis=1, ignore_index=True)

,0,1,2,3
a,0,1,5,6
b,2,3,NaN,NaN
c,4,5,7,8


In [127]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])

In [129]:
df1

,a,b,c,d
0,0.622801,-0.028743,-0.688359,-0.119908
1,-0.566748,-1.901774,-0.034787,-0.694528
2,-1.938566,-0.980228,-0.650725,-0.240330


In [130]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [131]:
df2

,b,d,a
0,-0.313706,-0.922649,-0.194936
1,-1.234290,-0.442316,-0.061039


In [132]:
pd.concat([df1, df2])

,a,b,c,d
0,0.622801,-0.028743,-0.688359,-0.119908
1,-0.566748,-1.901774,-0.034787,-0.694528
2,-1.938566,-0.980228,-0.650725,-0.240330
0,-0.194936,-0.313706,NaN,-0.922649
1,-0.061039,-1.234290,NaN,-0.442316


In [133]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.622801,-0.028743,-0.688359,-0.119908
1,-0.566748,-1.901774,-0.034787,-0.694528
2,-1.938566,-0.980228,-0.650725,-0.240330
3,-0.194936,-0.313706,NaN,-0.922649
4,-0.061039,-1.234290,NaN,-0.442316


In [134]:
pd.concat([df1, df2], axis=1)

,a,b,c,d,b,d,a
0,0.622801,-0.028743,-0.688359,-0.119908,-0.313706,-0.922649,-0.194936
1,-0.566748,-1.901774,-0.034787,-0.694528,-1.234290,-0.442316,-0.061039
2,-1.938566,-0.980228,-0.650725,-0.240330,NaN,NaN,NaN


In [135]:
pd.concat([df1, df2], axis=1, ignore_index=True)

,0,1,2,3,4,5,6
0,0.622801,-0.028743,-0.688359,-0.119908,-0.313706,-0.922649,-0.194936
1,-0.566748,-1.901774,-0.034787,-0.694528,-1.234290,-0.442316,-0.061039
2,-1.938566,-0.980228,-0.650725,-0.240330,NaN,NaN,NaN


In short, concatenating with `axis=0` is similar to **R**'s `rbind` and `axis=1` similar to `cbind`.

### Combining Data with Overlap
Another data combination situation can't be expressed as either a merge or concatenation operation. You may have two datasets whose indexes overlap in full or part. As a motivating example, consider  NumPy's `where` function, which expressed a vectorized if-else.

In [136]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], 
              index=['f', 'e', 'd', 'c', 'b', 'a'])

In [137]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [138]:
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])

In [139]:
b

f    0
e    1
d    2
c    3
b    4
a    5
dtype: float64

In [140]:
np.where(pd.isnull(a), b, a)

array([ 0. ,  2.5,  2. ,  3.5,  4.5,  5. ])

Series has a `combine_first` method, which performs the equivalent of this operation plus data alignment.

In [141]:
b.combine_first(a)

f    0
e    1
d    2
c    3
b    4
a    5
dtype: float64

In [142]:
a.combine_first(b)

f    0.0
e    2.5
d    2.0
c    3.5
b    4.5
a    5.0
dtype: float64

Now, a DataFrames example of `combine_first`

In [148]:
df1 = pd.DataFrame({'a': [1, np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})

In [149]:
df1

,a,b,c
0,1,NaN,2
1,NaN,2,6
2,5,NaN,10
3,NaN,6,14


In [150]:
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

In [151]:
df2

,a,b
0,5,NaN
1,4,3
2,NaN,4
3,3,6
4,7,8


In [152]:
df1.combine_first(df2)

,a,b,c
0,1,NaN,2
1,4,2,6
2,5,4,10
3,3,6,14
4,7,8,NaN


`combine_first` says: if the element of the calling dataframe is `false` then substitute with passed dataframes value, for each column.

## Reshaping and Pivoting

### Reshaping with Hierarchical Indexing
Two primary actions are:
    * stack - pivots from columns to rows
    * unstack - pivots from rows into columns

In [153]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))

In [154]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [155]:
# pivot from column to row
data.stack()

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [158]:
result = data.stack()

In [159]:
result.unstack() # by default unstacks inner most level (number)

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [161]:
result.unstack(0) # unstack outer most ('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [162]:
result.unstack('state') # can also refer to name of index

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [163]:
result.unstack('number') # same as default

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Unstacking might introduce missing data if all of the values in the level aren't found in each of the subgroups.

In [164]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])

In [165]:
s1

a    0
b    1
c    2
d    3
dtype: int64

In [166]:
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

In [167]:
s2

c    4
d    5
e    6
dtype: int64

In [168]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [169]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [170]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,NaN
two,NaN,NaN,4,5,6


In [171]:
# stacking filters out missing data by default, so the operation is easily invertible
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: float64

In [172]:
data2.unstack().stack(dropna=False)

one  a     0
     b     1
     c     2
     d     3
     e   NaN
two  a   NaN
     b   NaN
     c     4
     d     5
     e     6
dtype: float64

In [173]:
data2.unstack(0).stack(dropna=False)

a  one     0
   two   NaN
b  one     1
   two   NaN
c  one     2
   two     4
d  one     3
   two     5
e  one   NaN
   two     6
dtype: float64

In [174]:
data2.unstack(0).stack()

a  one    0
b  one    1
c  one    2
   two    4
d  one    3
   two    5
e  two    6
dtype: float64

When unstacking in a DataFrame, the level unstacked becomes the lowest level in the result.

In [176]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,NaN
two,NaN,NaN,4,5,6


In [180]:
df = pd.DataFrame({'left': result,
                   'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))

In [181]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [182]:
df.index

MultiIndex(levels=[[u'Ohio', u'Colorado'], [u'one', u'two', u'three']],
           labels=[[0, 0, 0, 1, 1, 1], [0, 1, 2, 0, 1, 2]],
           names=[u'state', u'number'])

In [183]:
# inner most level is number, which is unstacked. It now becomes the lowest level in the resulting dataframe
df.unstack()

side     left           right          
number    one two three   one two three
state                                  
Ohio        0   1     2     5   6     7
Colorado    3   4     5     8   9    10

In [184]:
# unstack state
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

### Pivoting "long" to "wide" format
A common way to store multiple time series in databases and CSV is in so-called *long* or *stacked* format.

In [185]:
# import requests and string io library
from StringIO import StringIO
import requests

In [186]:
# link to data
data_link = "https://raw.githubusercontent.com/wesm/pydata-book/master/ch07/macrodata.csv"

In [187]:
string = requests.get(data_link).content

In [190]:
df_from_string = pd.read_csv(StringIO(string.decode('utf-8')))

In [199]:
df_from_string.ix[:10, ['quarter']] # dataframe

,quarter
0,1
1,2
2,3
3,4
4,1
5,2
6,3
7,4
8,1
9,2


In [201]:
df_from_string.ix[:10, 'quarter'] # series

0     1
1     2
2     3
3     4
4     1
5     2
6     3
7     4
8     1
9     2
10    3
Name: quarter, dtype: float64

Now, back to the example from the book

In [203]:
periods = pd.PeriodIndex(year=df_from_string.year,
                         quarter=df_from_string.quarter,
                         name='date')

In [204]:
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='int64', name=u'date', length=203, freq='Q-DEC')

In [211]:
df_from_string = pd.DataFrame(df_from_string.to_records(),
                              columns=pd.Index(['realgdp', 'infl', 'unemp'], name='item'),
                              index=periods.to_timestamp('D', 'end'))

In [212]:
ldata = df_from_string.stack().reset_index().rename(columns={0: 'value'})

In [217]:
ldata[:10]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204
